In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#%%capture
DATA_DIR = "/content/drive/My Drive/nlp_frameworks/deeppavlov/movie_sentiment_new"
CONFIG_NAME = f"{DATA_DIR}/movie_sentiment_bert_new.json"
! pip install deeppavlov
! pip install tensorflow-gpu==1.15.2
! python -m deeppavlov install "$CONFIG_NAME"
! python -m deeppavlov download "$CONFIG_NAME"

In [ ]:
# splitting orig for train and valid

# import pandas as pd
# train_src = pd.read_csv(f"{DATA_DIR}/data/train_src.csv")
# row_num = len(train_src.index)
# train_src = train_src.sample(frac=1)
# train_ds = train_src.head(int(row_num*0.9))
# valid_ds = train_src.tail(int(row_num*0.1))
# train_ds.to_csv(f"{DATA_DIR}/data/train.csv", index=None)
# valid_ds.to_csv(f"{DATA_DIR}/data/valid.csv", index=None)

In [ ]:
#%%capture
from deeppavlov import build_model, configs
model = build_model(CONFIG_NAME, download=True)

In [ ]:
from deeppavlov import train_model
model = train_model(CONFIG_NAME)

In [ ]:
import pandas as pd
import numpy as np

test = pd.read_csv(f"{DATA_DIR}/data/test.csv")
x = test.Phrase.values
batch_size = 100
max_ind = x.shape[0] // batch_size if x.shape[0] % batch_size == 0 else x.shape[0] // batch_size + 1
predicted = None
for i in range(max_ind):
    index_first = i * batch_size

    if x.shape[0] % batch_size == 0 or i != max_ind - 1:
        index_last = index_first + batch_size
    else:
        index_last = index_first + x.shape[0] % batch_size

    pred_ = model(x[index_first:index_last])

    if predicted is not None:
        predicted = np.concatenate((predicted, pred_), axis=0)
    else:
        predicted = y_probability

#predicted = model(test.Phrase.values)
subm = pd.read_csv(f"{DATA_DIR}/data/sampleSubmission.csv")
subm['Sentiment'] = predicted
subm.to_csv(f"{DATA_DIR}/data/subm.csv", index=None)